# Joint hypocenter-velocity inversion

Note: most of the following is coming from the thesis of Lisa Block (1991)

The nonlinear joint hypocenter-velocity inversion problem is solved by iterating the linearized problem, following the separation of parameters technique described by Pavlis and Booker (1980).  Three types of constraints are applied to the velocity models. To prevent wild fluctuations in the velocity structures at poorly resolved nodes, a smoothness constraint is added. This is accomplished by minimizing the spatial velocity derivatives in addition to the arrivai time residuals. Another constraint is applied to keep the velocities within specified bounds. It is implemented by applying a penalty for velocities which fall outside the desired range, and then minimizing the penalties during the subsequent iteration. With the last type of velocity constraint, velocities computed from independent surveys are used to directly constrain the velocity model at certain points within the grid.

## Formulation

Let $t_o$ be an observed arrivai time (either P or S); $t_c$ = the calculated arrivai time, based on initial estimates of the velocity structure, earthquake location and origin time, and station correction; and $r$ the residual, i.e. $r - t_o - t_c$.  The goal is to change the model parameters so that the new calculated arrivai time, $t_c$ + $\Delta t_c$ is equal to the observed arrivai time

$\Delta t_c = t_o - t_c = r$

